# Ερώτηση 4

In [6]:
import pandas as pd
import numpy as np
import json
import time 

dataset_path = '../yelp-dataset/' # Change this path to match your local dataset folder path

business_filename = 'yelp_academic_dataset_business.json'
users_filename = 'yelp_academic_dataset_user.json'

In [7]:
list_business = []
with open(dataset_path + business_filename, 'r', encoding="utf8") as buisness_file:
    for line in buisness_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 10:
            list_business.append([json_dict['business_id'], json_dict['name'], json_dict['review_count']]) 

In [8]:
df_buisness = pd.DataFrame(list_business, columns=['business_id', 'name', 'reviews_count'])
display(df_buisness)

,business_id,name,reviews_count
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,36
1,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,26
2,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,38
3,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,81
4,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,18
...,...,...,...
103798,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,13
103799,hskVqZCPqy-omm9CHi44xQ,Domino's,16
103800,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,217
103801,PotiAhtDMYMNsJCzV0oLAw,Walmart,13


In [11]:
list_users = []
with open(dataset_path + users_filename, 'r', encoding="utf8") as users_file:
    for line in users_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 10:
            list_users.append([json_dict['user_id'], json_dict['name'], json_dict['review_count'], json_dict['friends']]) 

In [12]:
df_users = pd.DataFrame(list_users, columns=['business_id', 'name', 'reviews_count', 'friends'])
display(df_users)

 SP3A9cFVGJcR1oZoE3qr8w, EWi15aig7njPARqGoBOttg, MSHLx8DN5TyBGnGskmasKA, 6FkhvdB4NxnrvnXQB-V5hw, vSFEkld3MjA-Ltg_z69nuQ, kCIgy39IkJNIEYGHH-fbJg, Njxe9Qr6IGbPyp2MkdGQ9w, 15lpaZIh72pwP6lixe6jhQ, y0ymXObCpUU8tPVUiT87fQ, e7mC62NvH9kyGG6YM84cAg, kuqoLJkl4JKnoT5L9OJ5Eg, rgUOXqgL9d6haySNGbubXA, KpDEz_fkNXzxgLMP-N8TrA, MdbdNXBAyhsqaY4kUXuM2w, suQ9LddlGxOwxry0OEPm8Q, 7-voc0AP-hr7D2WmBOTBQw, Ue2ci1z8GbnX4ChnZa7gSQ, hZLl2A68yMcLEKrADHmE3Q, w5LuMkYLijWU0okkilWzqA, m1IYrFif8u7Tn1VYRDmZvA, _ts72RvDIOCejrJWIeIxZg, o-tNWwTeRe_qVN0EHRDDww, CcRfRjqvHkMTWK7aU4bnPw, rlBlz1s7oSGn4zM5QlO_xA, C-jwnYAY10RsZ5BR_M6_fA, HEk-W4yKdlrv3cMJL91Ggw, 6xh67hxk6ufr0PtG0BI8_Q, ImUjiY8IsHWYuVidVr38Jw, ZMMOMP5kLoXWcm3keolHCA, hAqVXdwnlUYxdIANvqqG9A, bWFiL1wkiCCoHEKK7GN2gA, jBfYJzsFEOk3zvOPxn0Kuw, Mp5gxHPkPLZqiXyAi8UtUA, VoO-eDoNC87qolK8Fz9yPA, duZ4ZLBu_mjHP2JLNzrAvw, YwgGrxpuKlxetOWhY1Nguw, GcJcrRY_oN9zrDTXshrdqQ, FpB44ccQnPntnZQmWTTKDA, owCLHV8XxI4Uj1yOGf1FHA, LFiLbQ6rNNjHS3mWwDmKEA, ydIdyGQyO8eZe0yqgs0DfA, 8KGRYbh5WJzUmne